In [28]:
import pytesseract
import cv2
from PIL import Image 
import re
import pandas as pd
from PIL import Image
import fitz
import os

# Code to Convert PDF to PNG

In [26]:
def convert_pdf2img(input_file: str,pages=None):
    """Converts pdf to image and generates a file by page"""
    # Open the document
    pdfIn = fitz.open(input_file)
    output_files = []
    # Iterate throughout the pages
    for pg in range(pdfIn.pageCount):
        if str(pages) != str(None):
            if str(pg) not in str(pages):
                continue
        # Select a page
        page = pdfIn[pg]
        rotate = int(0)
        # PDF Page is converted into a whole picture 1056*816 and then for each picture a screenshot is taken.
        # zoom = 1.33333333 -----> Image size = 1056*816
        # zoom = 2 ---> 2 * Default Resolution (text is clear, image text is hard to read)    = filesize small / Image size = 1584*1224
        # zoom = 4 ---> 4 * Default Resolution (text is clear, image text is barely readable) = filesize large
        # zoom = 8 ---> 8 * Default Resolution (text is clear, image text is readable) = filesize large
        zoom_x = 2
        zoom_y = 2
        # The zoom factor is equal to 2 in order to make text clear
        # Pre-rotate is to rotate if needed.
        mat = fitz.Matrix(zoom_x, zoom_y).preRotate(rotate)
        pix = page.getPixmap(matrix=mat, alpha=False)
        path = 'Makro Invoices/Makro Invoices/New/'
        output_file = f"{path+os.path.splitext(os.path.basename(input_file))[0]}_page{pg+1}.png"
        pix.writePNG(output_file)
        output_files.append(output_file)
    pdfIn.close()
    summary = {
        "File": input_file, "Pages": str(pages), "Output File(s)": str(output_files)
    }
    # Printing Summary
    print("## Summary ########################################################")
    print("\n".join("{}:{}".format(i, j) for i, j in summary.items()))
    print("###################################################################")
    return output_files

In [29]:
convert_pdf2img('Makro Invoices/Makro Invoices/Convert/PEDIDO MENEGHELLO 4000107017--COCINA MINAMI.pdf')

Deprecation: 'preRotate' removed from class 'Matrix' after v1.19 - use 'prerotate'.
Deprecation: 'getPixmap' removed from class 'Page' after v1.19 - use 'get_pixmap'.
Deprecation: 'writePNG' removed from class 'Pixmap' after v1.19 - use 'save'.


## Summary ########################################################
File:Makro Invoices/Makro Invoices/Convert/PEDIDO MENEGHELLO 4000107017--COCINA MINAMI.pdf
Pages:None
Output File(s):['Makro Invoices/Makro Invoices/New/PEDIDO MENEGHELLO 4000107017--COCINA MINAMI_page1.png']
###################################################################


['Makro Invoices/Makro Invoices/New/PEDIDO MENEGHELLO 4000107017--COCINA MINAMI_page1.png']

In [2]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [3]:
formats = pd.read_excel('invoice_formats.xlsx')
final = pd.read_excel('Extracted_Data.xlsx')

In [21]:
Ima = "Makro Invoices/Makro Invoices/New/PEDIDO MENEGHELLO 4000107454--COCINA BEACH_page1.png"

In [24]:
%%time
dat = {
    "Vat Number":'',
       "Total":0,
       "Date":'',
      "Invoice_Number":''}
img  = cv2.imread(Ima, cv2.COLOR_RGB2GRAY)
text=(pytesseract.image_to_string(img))
text_list = text.split("\n")
vat = []
date = []
print(type(text_list))
for idx, a in enumerate(text_list):
    if(a.strip() != ""):


        if (re.search("[A-Z]{1}-[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
            print(b)
        elif (re.search("[0-9]{8}-[A-Z]{1}",a)):
            b = re.findall("[0-9]{8}-[A-Z]{1}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
        elif (re.search("[0-9]{8}[A-Z]{1}",a)):
            b = re.findall("[0-9]{8}[A-Z]{1}",a)
            dat["Vat Number"] = b
            vat.append(b)
    
        elif (re.search("[A-Z]{1}-[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)

        elif (re.search("[A-Z]{1}[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
            
        elif (re.search("[A-Z]{1}[0-9]{2}/[0-9]{6}",a)):
            b = re.findall("[A-Z]{1}[0-9]{2}/[0-9]{6}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
            
        elif (re.search("[A-Z]{1}-[0-9]{2}.[0-9]{3}.[0-9]{3}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{2}.[0-9]{3}.[0-9]{3}",a)
            dat["Vat Number"] = b[0]
        elif (re.search("[A-Z]{1}-[0-9]{2}/[0-9]{6}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{2}/[0-9]{6}",a)
            dat["Vat Number"] = b[0]

        if len(vat)>1:
            dat["Vat Number"] = vat[0]
        else:
            pass
         #Date section           
        if (re.search("\d{2}/\d{2}/\d{4}",a)):
            b  = re.findall("\d{2}/\d{2}/\d{4}", a)
            dat["Date"] = b[0]
            date.append(b[0])
        elif (re.search("\d{2}-\d{2}-\d{4}",a)):
            b = re.findall("\d{2}-\d{2}-\d{4}",a)
            dat["Date"] = b[0]
            date.append(b[0])
        elif (re.search("\d{2}/\d{2}/\d{2}",a)):
            b = re.findall("\d{2}/\d{2}/\d{2}",a)
            dat["Date"] = b[0]
            date.append(b[0])
        elif (re.search("\d{1}/\d{2}/\d{2}",a)):
            b = re.findall("\d{1}/\d{2}/\d{2}",a)
            dat["Date"] = b[0]
            date.append(b[0])
        
        elif (re.search("[0-9]{2}\.[0-9]{2}\.[0-9]{4}",a)):
            b = re.findall("[0-9]{2}\.[0-9]{2}\.[0-9]{4}",a)
            dat["Date"] = b[0]
            date.append(b[0])
    
        if len(date)>1:
            dat["Date"] = date[0]

        #invoice number section
        if ("Factura" in a or "ALBARAN" in a or "Factura: " in a):

            print(a)
            b = a.split(" ")
            if b[0] == "ALBARAN":
                dat["Invoice_Number"] = b[1]
            print(b)
            if len(b)>3:
                print(b)
                if len(b)>4 and b[2] !='—' and len(b[1])!=25 and len(b[1])>5:
                    dat["Invoice_Number"] = b[1]+b[2]
                    print(b)
                elif len(b)>=5 and len(b[1])==25:
                    dat["Invoice_Number"] = b[1]
                elif len(b)>6 and len(b[3])>15:
                    dat["Invoice_Number"] = b[1]+ b[2]+ b[3]
                elif len(b)>6 and b[1]== "0/0":
                    dat["Invoice_Number"] = b[1] + b[2]
                    
        if ("Tarjeta" in a or "Credito" in a or  "pagina" in a or "Total:" in a  or "TOTAL" in a or  "Total factura" in a or  "Total Factura" in a or "TOTAL FACTURA" in a or "Total CON IGIC" in a  or "IMPORTE FACTURA EU" in a or "IMPORTE FACTURA" in a or "EUROS" in a or "Importe:" in a):
            print("Case Total")
            b = a.split(" ")
            print(b)
            if len(b[-1])==2:
                
                dat["Total"] = b[-2]+b[-1]
                print(b[-2]+b[-1])
                
            if b[-1]=='€':
                dat["Total"] = b[-2]
            if b[-1] == "|":
                dat["Total"] = b[-2]
            if b[-1] == "EUR":
                dat["Total"] = b[-2]
            else:
                dat["Total"] = b[-1]
                print(b[-1])
                
if dat["Total"] == 0 or len(dat['Total'])==1 or dat["Total"]=="SUB TOTAL"  or dat["Total"] == 'factura' or dat["Total"]=="FACTURA" or dat["Total"]== "EUROS" or dat["Total"] == "©" or dat["Total"]=="TOTAL":
        tot = []
        im = Image.open(Ima) 
        # Size of the image in pixels (size of orginal image) 
        # (This is not mandatory) 
        width, height = im.size 
        print(im.size)
        # Setting the points for cropped image 
        left = 605
        top = 600
        right = 1675
        bottom = 1590
        # Cropped image of above dimension 
        # (It will not change orginal image) 
        im1 = im.crop((left, top, right, bottom)) 
         # Shows the image in image viewer 
        im1.save("Makro Invoices/Makro Invoices/New/pp2.jpg")
        img  = cv2.imread('Makro Invoices/Makro Invoices/New/pp2.jpg', cv2.COLOR_RGB2GRAY)
        tex=(pytesseract.image_to_string(img,config='outputbase digits'))
        text_list = tex.split("\n")
        text_list = text_list
        print(text_list)
        for i in text_list:
            if len(i)>=4 and len(i)<=8:
                print(i)
                tot.append(i)
                dat["Total"] = max(tot)
            
            
        

<class 'list'>
Case Total
['Bruto:', '24,25', 'Reducc:', '0,00', 'Portes/Fletes:', '0,00', 'Cargos:', '0,00', 'Importe:', '24,25']
24,25
Wall time: 5.86 s


In [25]:
dat

{'Vat Number': ['B45010808'],
 'Total': '24,25',
 'Date': '30.06.2022',
 'Invoice_Number': ''}

In [12]:
data  =  pd.DataFrame(dat,index=[0])
data['Vat Number']= data['Vat Number'].str.replace(' ', '')
print(data)

  Vat Number   Total        Date Invoice_Number
0  B45010808  185,09  1000005010               


In [13]:
data['Vat Number']= data['Vat Number'].str.replace('-', '')
data['Vat Number']= data['Vat Number'].str.replace(',', '')
data['Vat Number']= data['Vat Number'].str.replace('.', '')
data

F:\anaconda3\envs\python_3_7\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


,Vat Number,Total,Date,Invoice_Number
0,B45010808,"185,09",1000005010,


In [14]:
formats['last']= (formats['nif'].isin(data['Vat Number']))

In [15]:
df2 = formats.query("last == True")

In [16]:
df2

,formato,nombre,nif,last
9341,\d\d\d/\d\d\d\d\d\d,FIESTA HOTELES RESORT SL,B45010808,True
9342,[A-Z]-[A-Z][A-Z][A-Z]-\d\d\d-\d\d-[A-Z]-\d\d\d...,FIESTA HOTELES RESORT SL,B45010808,True


In [28]:
inv = 0
for i in df2['formato']:
    inv = i
pat = 'F1-1-01353'
factura = re.compile('[0-9]{10}')
mo = factura.search(text)
dat["Invoice_Number"]=mo.group()

In [33]:

dat["Date"]='30.06.2022'
dat

{'Vat Number': ['B45010808'],
 'Total': '185,09',
 'Date': '30.06.2022',
 'Invoice_Number': '1000005010'}

In [32]:
text

"PALLADIUM\n\nHOTEL GROUP\n\nFiesta Hotels & Resorts, S.L\nAvda. Bartolomé Rosellé 18\n07800 Ibiza\n\nESPANA\n\nTelf. 971313811 Fax.\nB45010808\n\nDireccion de envio\nHotel Ushuaia Ibiza\nPlaya d'en Bossa\n\nDireccion de pedido\n1000005010 - HERMANOS MENEGHELLO, S.L.\nCTRA SAN ANTONIO KM 1.200\n\n07817 Sant Jordi de Ses Salines 07800 IBIZA\nEspafia ESPANA\nTelf. Fax. Telf. Fax.\nPasteleria Email:\nLN.ILF. B57170300\nN° documento: 4000107007 Moneda: EUR\n\nFecha documento: 30.06.2022\nProveedor:\n\nHERMANOS MENEGHELLO, S.L.\n\nCondiciones de pago: dentro de los 60 dias\nIncoterms:\n\nMaterial Descripcién\n\nMaterial\n\nCantidad Precio/UM Reducc Imp. Neto Fecha\n\nPAN ORO 8X8 25UD\nRef 21000010. Sosa COD 36283\n\n1068328\n\nEMULSIONANTE SOJA WHIP\nSOSA 300GR\n\nRef 00200524. Sosa 39028\n\n1098053\n\nEMULSIONANTE POTATO WHIP.\nSOSA 400GR\n\nRef 00200531. Sosa 38967\n\n1098054\n\nMELOCOTON CONFITADO A\nDADOS 1,5KG\nRef 44200147. Sosa\n\n1098064\n\nCOLORANTE BLANCO LACA\nPOLVO 60GR\nRef 593

In [20]:
import boto3
import io

In [21]:
im = Image.open(Ima)

In [22]:
buffered = io.BytesIO()
im.save(buffered, format='PNG')
client = boto3.client('textract')
response = client.analyze_document(
    Document={'Bytes': buffered.getvalue()},
    FeatureTypes=['TABLES']
)

In [23]:
def map_blocks(blocks, block_type):
    return {
        block['Id']: block
        for block in blocks
        if block['BlockType'] == block_type
    }

blocks = response['Blocks']
tables = map_blocks(blocks, 'TABLE')
cells = map_blocks(blocks, 'CELL')
words = map_blocks(blocks, 'WORD')
selections = map_blocks(blocks, 'SELECTION_ELEMENT')

def get_children_ids(block):
    for rels in block.get('Relationships', []):
        if rels['Type'] == 'CHILD':
            yield from rels['Ids']
            
dataframes = []

for table in tables.values():

    # Determine all the cells that belong to this table
    table_cells = [cells[cell_id] for cell_id in get_children_ids(table)]

    # Determine the table's number of rows and columns
    n_rows = max(cell['RowIndex'] for cell in table_cells)
    n_cols = max(cell['ColumnIndex'] for cell in table_cells)
    content = [[None for _ in range(n_cols)] for _ in range(n_rows)]

    # Fill in each cell
    for cell in table_cells:
        cell_contents = [
            words[child_id]['Text']
            if child_id in words
            else selections[child_id]['SelectionStatus']
            for child_id in get_children_ids(cell)
        ]
        i = cell['RowIndex'] - 1
        j = cell['ColumnIndex'] - 1
        content[i][j] = ' '.join(cell_contents)

    # We assume that the first row corresponds to the column names
    dataframe = pd.DataFrame(content[1:], columns=content[0])
    dataframes.append(dataframe)

In [26]:
if len(dataframes)>1:
    if  "Concepto" in dataframes[1] or "MM Num. articulo" in dataframes[1] or "PRECIO" in dataframes[1]  or  "Precio/UM" in dataframes[1] or "Precio" in dataframes[1] or "PRECIO/UD" in dataframes[1] or "PRECIO UD." in dataframes[1] or "Num. artículo" in dataframes[1] or "Código" in dataframes[1] or "FECHA" in dataframes[1] or  "ARTICULO" in dataframes[1] or  "PRECIO KG" in dataframes[1] or "CODIGO" in dataframes[1] or "Cod." in dataframes[1] or "ARTÍCULO" in dataframes[1] or "Tipo" in dataframes[1] or "PRECIO UD"  in dataframes[1]:
        df_t = pd.DataFrame(dataframes[1])
        df_t.to_excel('Makro Invoices/Makro Invoices/Saved_Datad.xlsx')
        print("Yes 1")

if "Concepto" in dataframes[0] or "MM Num. articulo" in dataframes[0] or "PRECIO" in dataframes[0] or  "Precio/UM" in dataframes[0]  or "Precio" in dataframes[0] or  "PRECIO/UD" in dataframes[0] or "Código" in dataframes[0] or "PRECIO UD." in dataframes[0]  or "FECHA" in dataframes[0] or "ARTICULO" in dataframes[0] or "PRECIO KG" in dataframes[0] or "CODIGO" in dataframes[0] or "Cod." in dataframes[0] or "ARTÍCULO" in dataframes[0] or "Tipo" in dataframes[0] or "PRECIO UD"  in dataframes[0]:
        df_t = pd.DataFrame(dataframes[0])
        df_t.to_excel('Makro Invoices/Makro Invoices/Saved_Datad.xlsx')
        print("Yes 2")
if len(dataframes)>2:
    if "Concepto" in dataframes[2] or "MM Num. articulo" in dataframes[2] or  "PRECIO" in dataframes[2] or  "Precio/UM" in dataframes[2] or "Precio" in dataframes[2] or  "PRECIO/UD" in dataframes[2] or "Código" in dataframes[2] or "PRECIO UD" in dataframes[2] or "PRECIO UD." in dataframes[2]  or "FECHA" in dataframes[2] or "ARTICULO" in dataframes[2] or "PRECIO KG" in dataframes[2] or "CODIGO" in dataframes[2] or "Cod." in dataframes[2] or "ARTÍCULO" in dataframes[2] or "Tipo" in dataframes[2]:
        df_t = pd.DataFrame(dataframes[2])
        df_t.to_excel('Makro Invoices/Makro Invoices/Saved_Datad.xlsx')
        print("Yes 3")


Yes 1


In [27]:
df_t

,Material,Descripción,Material,Cantidad,Precio/UM,Reducc,Imp. Neto,Fecha
0,1068328,PAN ORO 8X8 25UD Ref 21000010. Sosa COD 36283,,"1,00","77,00/1UN","0,00","77,00",
1,1098053,EMULSIONANTE SOJA WHIP SOSA 300GR Ref 00200524...,,"1,00","30,16/1UN","0,00","30,16",30.06.2022
2,1098054,EMULSIONANTE POTATO WHIP SOSA 400GR Ref 002005...,,"1,00","35,23/1UN","0,00","35,23",30.06.2022
3,1098064,"MELOCOTON CONFITADO A DADOS 1,5KG Ref 44200147...",,"1,00","20,80/1UN","0,00","20,80",30.06.2022
4,1085030,COLORANTE BLANCO LACA POLVO 60GR Ref 59300017....,,"5,00","4,38/1UN","0,00","21,90",30.06.2022
5,,,,,,,,01.07.2022


In [498]:
dat

{'Vat Number': 'B38574281',
 'Total': '10.00',
 'Date': '03/07/2020',
 'Invoice_Number': 'F1-1-01353'}

In [499]:
json_f = {}
json_f['file'] = Ima
json_f['labels'] = dat
json_f['tables'] = df_t
print(json_f)

{'file': 'Makro Invoices/Makro Invoices/Converted/03.07.20_Extintores Valle Union Sl_3243,8_page1.png', 'labels': {'Vat Number': 'B38574281', 'Total': '10.00', 'Date': '03/07/2020', 'Invoice_Number': 'F1-1-01353'}, 'tables':   Código                       Descripción LOTE Cantidad Precio DTO   Importe
0  04625             RACORD MANGUERA 25 MM         25,00   6,86        171,50
1  04134           RACOR MANGUERA 45 ALUM.         25,00   6,30        157,50
2  04397                RACOR R/ INT 25 MM         35,00   6,30        220,50
3  04396              RACORD R/EXT. 25 MM.         25,00   6,30        157,50
4  04237                RACORD R/INT 45 MM         25,00   8,96        224,00
5  04322      VALVULA LATON C/RACORD 25 MM         10,00  21,28        212,80
6    SIN                            RACORD                                   
7  02501  EXTINTOR 5 KG. CO2 "VALLE-UNION"         60,00  35,00      2.100,00
8                           BULTO: 1 PALET                               

In [34]:
dat

{'Vat Number': ['B45010808'],
 'Total': '185,09',
 'Date': '30.06.2022',
 'Invoice_Number': '1000005010'}